In [71]:
import cv2
import keras
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import seaborn as sns


from skimage.io import imread
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2

In [72]:
# Cargo el modelo entrenado en predictor_caras.ipynb
model = load_model('modelo_entrenado.keras') 

In [73]:

ROOT_PATH = './data/images/'
TRAIN_PATH = ROOT_PATH + "train/"
TEST_PATH = ROOT_PATH + "test/"

train_csv_path = 'train_set.csv'
test_csv_path = 'test_set.csv'
sample_submission_path = 'sample_submision.csv'

train_df = pd.read_csv(train_csv_path)
test_df = pd.read_csv(test_csv_path)
sample_submission_df = pd.read_csv(sample_submission_path)

In [74]:
def replace_backslash(path):
    return path.replace('\\', '/')

# Aplicar la función a la columna 'path'
train_df['path'] = train_df['path'].apply(replace_backslash)

In [75]:
def read_train_data(dataframe, reshape_dim=(64, 64)):
    X = []
    y = []
    for index, row in dataframe.iterrows():
        img_path = os.path.join(ROOT_PATH, row['path'].replace('../data/images/', ''))
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if image is not None:
            image = cv2.resize(image, reshape_dim) 
            X.append(image)
            y.append(row['label'])
    
    X = np.array(X).reshape(-1, reshape_dim[0], reshape_dim[1], 1) 
    y = np.array(y)
    return X, y

def read_test_data(dataframe, reshape_dim=(64, 64)):
    X = []
    for index, row in dataframe.iterrows():
        img_path = os.path.join(TEST_PATH, f"{row['id_img']}.jpg")
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE) 
        if image is not None:
            image = cv2.resize(image, reshape_dim) 
            X.append(image)
    
    X = np.array(X).reshape(-1, reshape_dim[0], reshape_dim[1], 1)
    return X

In [76]:
X_train, y_train = read_train_data(train_df)
X_test = read_test_data(test_df)

In [77]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [78]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)

In [79]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [80]:
model.summary()

Model: "sequential_39"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_228 (Conv2D)             │ (None, 64, 64, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_227         │ (None, 64, 64, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_229 (Conv2D)             │ (None, 64, 64, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_228         │ (None, 64, 64, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_131               │ (None, 32, 32, 64)     │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_164 (Dropout)           │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_230 (Conv2D)             │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_229         │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_231 (Conv2D)             │ (None, 32, 32, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_230         │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_132               │ (None, 16, 16, 128)    │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_165 (Dropout)           │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_232 (Conv2D)             │ (None, 16, 16, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_231         │ (None, 16, 16, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_233 (Conv2D)             │ (None, 16, 16, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_232         │ (None, 16, 16, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_133               │ (None, 8, 8, 256)      │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_166 (Dropout)           │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_234 (Conv2D)             │ (None, 8, 8, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_233         │ (None, 8, 8, 512)      │         2,048 │
│ (BatchNormalization)            │                        │             

 Total params: 42,407,255 (161.77 MB)

 Trainable params: 14,134,471 (53.92 MB)

 Non-trainable params: 3,840 (15.00 KB)

 Optimizer params: 28,268,944 (107.84 MB)

In [81]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
callbacks = [EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
             ModelCheckpoint('best_model_reentrenado.keras', save_best_only=True, monitor='val_loss')]

In [82]:
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_val, y_val), callbacks=[callbacks])

Epoch 1/50
361/361 ━━━━━━━━━━━━━━━━━━━━ 649s 2s/step - accuracy: 0.6947 - loss: 1.5673 - val_accuracy: 0.6182 - val_loss: 1.7942
Epoch 2/50
361/361 ━━━━━━━━━━━━━━━━━━━━ 626s 2s/step - accuracy: 0.7220 - loss: 1.5028 - val_accuracy: 0.6141 - val_loss: 1.8397
Epoch 3/50
361/361 ━━━━━━━━━━━━━━━━━━━━ 603s 2s/step - accuracy: 0.7264 - loss: 1.5041 - val_accuracy: 0.6160 - val_loss: 1.8472
Epoch 4/50
361/361 ━━━━━━━━━━━━━━━━━━━━ 623s 2s/step - accuracy: 0.7437 - loss: 1.4805 - val_accuracy: 0.6239 - val_loss: 1.8387
Epoch 5/50
361/361 ━━━━━━━━━━━━━━━━━━━━ 631s 2s/step - accuracy: 0.7499 - loss: 1.4945 - val_accuracy: 0.6095 - val_loss: 1.9206
Epoch 6/50
361/361 ━━━━━━━━━━━━━━━━━━━━ 616s 2s/step - accuracy: 0.7541 - loss: 1.5112 - val_accuracy: 0.6168 - val_loss: 1.9238
Epoch 7/50
361/361 ━━━━━━━━━━━━━━━━━━━━ 577s 2s/step - accuracy: 0.7633 - loss: 1.4987 - val_accuracy: 0.6208 - val_loss: 1.9377
Epoch 8/50
361/361 ━━━━━━━━━━━━━━━━━━━━ 575s 2s/step - accuracy: 0.7648 - loss: 1.5218 - val_accu

In [83]:
predictions = model.predict(X_test)
predictions = np.argmax(predictions, axis=1)

221/221 ━━━━━━━━━━━━━━━━━━━━ 40s 179ms/step


In [84]:
submission_df = pd.DataFrame({
    'id_img': test_df['id_img'],
    'label': label_encoder.inverse_transform(predictions)
})

In [85]:
submission_df.to_csv('submission7.csv', index=False)

print(submission_df.head())

   id_img  label
0   10052    sad
1   10065    sad
2   10079  angry
3   10095    sad
4   10121   fear
